### Load the data

In [1]:
import os
import mlflow

os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"
mlflow.set_experiment('Training_LSTM')

2023/10/24 18:57:12 INFO mlflow.tracking.fluent: Experiment with name 'Training_LSTM' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/779043405399066498', creation_time=1698166632525, experiment_id='779043405399066498', last_update_time=1698166632525, lifecycle_stage='active', name='Training_LSTM', tags={}>

### Create the Optuna Objective

In [2]:
import mlflow.keras
import optuna
from common import OptunaPruneCallback, mean_absolute_percentage_error, create_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import numpy as np


def objective(trial, data, coin):
    with mlflow.start_run() as run:
        mlflow.keras.autolog(log_models=False)

        # Define the search space for hyperparameters
        num_layers = trial.suggest_int('num_layers', 1, 4)
        units_per_layer = [trial.suggest_int(f'units_layer_{i}', 32, 256, 32) for i in range(num_layers)]
        sequence_length = trial.suggest_int('sequence_length', 5, 20)
        learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-3)
        dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
        min_max_scaling = trial.suggest_int('min_max_scaling', 0, 1)
        layer_type = trial.suggest_categorical('layer_type', ['LSTM', 'RNN'])

        layer_class = keras.layers.LSTM if layer_type == 'LSTM' else keras.layers.SimpleRNN 

        mlflow.log_params({
            'coin': coin,
            'num_layers': num_layers,
            'units_per_layer': units_per_layer,
            'sequence_length': sequence_length,
            'learning_rate': learning_rate,
            'dropout_rate': dropout_rate,
            'min_max_scaling': min_max_scaling
        })

        if min_max_scaling == 1:
            scaler = MinMaxScaler()
            data = scaler.fit_transform(np.array(data))

        X, y = create_sequences(data, sequence_length)

        # Build and compile the LSTM model
        model = keras.Sequential()
        for units in units_per_layer:
            model.add(layer_class(units, activation='relu', return_sequences=True, input_shape=(sequence_length, 1)))
            model.add(keras.layers.Dropout(dropout_rate))
        model.add(keras.layers.Dense(1))

        optimizer = keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=1.0)
        model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[mean_absolute_percentage_error])
        # Split the data into training and validation sets
        _X_train, X_val, _y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
        X_train, X_test, y_train, y_test = train_test_split(_X_train, _y_train, test_size=0.2, shuffle=False)

        # Train the model with early stopping
        history = model.fit(
            X_train,
            y_train,
            epochs=50,
            batch_size=32,
            validation_data=(X_val, y_val), 
            verbose=0,
            callbacks=[OptunaPruneCallback(trial=trial)]
        )

        # Evaluate the model on the validation set
        loss = model.evaluate(X_val, y_val)

        return loss[1]

/home/gianfranco/Desktop/uni/price-oracle/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-24 18:57:13.164807: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-24 18:57:13.168274: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-24 18:57:13.224220: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-24 18:57:13.224944: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in 

ImportError: cannot import name 'np' from 'numpy' (/home/gianfranco/Desktop/uni/price-oracle/venv/lib/python3.9/site-packages/numpy/__init__.py)

### Run the optimization

In [ ]:
import numpy as np

def evaluate_best_coin(coin, data, best_params):

    # Train the final model with the best hyperparameters
    # Define the search space for hyperparameters
    num_layers = best_params['num_layers']
    units_per_layer = [best_params[f'units_layer_{i}'] for i in range(num_layers)]
    sequence_length = best_params['sequence_length']
    learning_rate = best_params['learning_rate']
    dropout_rate = best_params['dropout_rate']
    layer_type = best_params['layer_type']

    layer_class = keras.layers.LSTM if layer_type == 'LSTM' else keras.layers.SimpleRNN 

    X, y = create_sequences(data, sequence_length)
    _X_train, X_val, _y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(_X_train, _y_train, test_size=0.2, shuffle=False)
    # concatenate train and validation sets
    X_train = np.concatenate((X_train, X_val))
    y_train = np.concatenate((y_train, y_val))

    with mlflow.start_run(run_name=f"Training_{coin}") as run:
        # Build and compile the LSTM model
        best_model = keras.Sequential()
        for units in units_per_layer:
            best_model.add(layer_class(units, activation='relu', return_sequences=True, input_shape=(sequence_length, 1)))
            best_model.add(keras.layers.Dropout(dropout_rate))
        best_model.add(keras.layers.Dense(1))

        optimizer = keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=1.0)
        best_model.compile(optimizer=optimizer, loss='mean_squared_error')

        best_model.fit(X_train, y_train, epochs=100, batch_size=32)

        from common import register_training_experiment
        preds = best_model.predict([X_test])
        register_training_experiment(y_test, preds)

In [ ]:
import numpy as np
from common import get_dataframe

# Normalize the data to the range [0, 1] to help the LSTM model converge faster

df = get_dataframe()

for coin in df.iloc[:, 1:]:
    data = np.array(df[coin]).reshape(-1, 1)

    # Create an Optuna study
    study_name = coin  # Unique identifier of the study.
    storage_name = "sqlite:///training.db"
    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    study = optuna.create_study(study_name=coin, direction='minimize', storage=storage_name, pruner=pruner)

    # Start the optimization process
    study.optimize(lambda trial: objective(trial, data, coin), n_trials=50)

    # Get the best hyperparameters
    try:
        best_params = study.best_params
        print("Best Hyperparameters:", best_params)
        evaluate_best_coin(coin, data, best_params)
    except ValueError:
        print("No best hyperparameters found")